In [1]:
ppdb_dict = None


In [ ]:
class TransformationDict(dictIn):
    '''
    A class storing a dictionary of phrasal, lexical,
    and syntactic transformations.
    '''
    def __init__(self):
        super(TransformationDict, self).__init__()
        self.index = {}
    
    def add(self, lhs, rhs):
        temp = self
        if len (lhs) == 0:
            return
        for i, token in enumerate(lhs):
            if token in temp:
                rule_list,temp= temp[token]
            else:
                rule_list = set()
                new_temp = TransformationDict()
                temp[token] = (rule_list, new_temp)
                temp = new_temp
            
            if 0 < i < len(lhs)-1:
                if token not in self.index:
                    self.index[token] = set()
                before = tuple(lhs[:i])
                after = tuple(lhs[i+1:])
                self.index[token].add((before,after))
            
        rule_list.add(rhs)
    
    def find_partial(self, partial):
        
                


In [1]:
import concurrent.futures
import gc

def process_chunk(chunk):
    """Process a chunk of the file."""
    parsed_data = []
    for line in chunk:
        columns = line.strip().split('|||')
        parsed_data.append(columns)
    return parsed_data

def read_in_chunks(file_object, chunk_size=1024*1024):
    """Lazy function to read a file piece by piece."""
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

def parse_file_parallel(file_path, chunk_size=1024*1024):
    # Open the file
    with open(file_path, 'r') as file:
        # Read and process the file in chunks
        with concurrent.futures.ProcessPoolExecutor() as executor:
            # Submit each chunk to the executor
            future_to_chunk = {executor.submit(process_chunk, chunk): chunk for chunk in read_in_chunks(file, chunk_size)}
            # Combine results as they are completed
            parsed_data = []
            for future in concurrent.futures.as_completed(future_to_chunk):
                parsed_data.extend(future.result())
                gc.collect()
    return parsed_data


In [2]:

data = parse_file_parallel(file, 512*512)
print(data[0])

In [8]:
def parse_file(file_path, cursor):

    # Open the file for reading
    with open(file_path, 'r') as file:
        # Read the file line by line
        
        for line in file:
            # Strip newline characters and split the line into columns
            columns = line.strip().split('|||')
            # Add the parsed columns to the parsed_data list
            cursor.execute("INSERT INTO ppdb VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", columns)

In [11]:
file = 'ppdb-2.0-xxxl-all'
data = parse_file(file)
print(data[0])

['[NN] ', ' transplant ', ' transplantation ', ' PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -logp(LHS|e1)=0.18597 -logp(LHS|e2)=0.14031 -logp(e1|LHS)=11.83583 -logp(e1|e2)=1.80507 -logp(e1|e2,LHS)=1.46728 -logp(e2|LHS)=11.47593 -logp(e2|e1)=1.49083 -logp(e2|e1,LHS)=1.10738 AGigaSim=0.63439 Abstract=0 Adjacent=0 CharCountDiff=5 CharLogCR=0.40547 ContainsX=0 Equivalence=0.371472 Exclusion=0.000344 GlueRule=0 GoogleNgramSim=0.03067 Identity=0 Independent=0.078161 Lex(e1|e2)=9.64663 Lex(e2|e1)=59.48919 Lexical=1 LogCount=4.67283 MVLSASim=NA Monotonic=1 OtherRelated=0.372735 PhrasePenalty=1 RarityPenalty=0 ForwardEntailment=0.177287 SourceTerminalsButNoTarget=0 SourceWords=1 TargetComplexity=0.98821 TargetFormality=0.98464 TargetTerminalsButNoSource=0 TargetWords=1 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=5.00000 WordLogCR=0 ', ' 0-0 ', ' OtherRelated']


In [16]:
import itertools
def parse_file(file_path, startLine, endLine):
    """
    Parse the file starting at startLine and ending at endLine.
    """
    # Create an empty list to store the parsed data
    parsed_data = []

    # Open the file for reading
    with open(file_path, 'r') as file:
        # Read the file line by line starting at line startLine
        for line in itertools.islice(file, startLine, endLine):
            # Strip newline characters and split the line into columns
            columns = line.strip().split('|||')
            # Add the parsed columns to the parsed_data list
            parsed_data.append(columns)
    
    return parsed_data
        

In [18]:
import pandas as pd
file = 'ppdb-2.0-xxxl-all'
data = parse_file(file,0,1000000)

# Create a DataFrame from the parsed data
# print(data[0])
df = pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment'])

df


['[NN] ', ' transplant ', ' transplantation ', ' PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -logp(LHS|e1)=0.18597 -logp(LHS|e2)=0.14031 -logp(e1|LHS)=11.83583 -logp(e1|e2)=1.80507 -logp(e1|e2,LHS)=1.46728 -logp(e2|LHS)=11.47593 -logp(e2|e1)=1.49083 -logp(e2|e1,LHS)=1.10738 AGigaSim=0.63439 Abstract=0 Adjacent=0 CharCountDiff=5 CharLogCR=0.40547 ContainsX=0 Equivalence=0.371472 Exclusion=0.000344 GlueRule=0 GoogleNgramSim=0.03067 Identity=0 Independent=0.078161 Lex(e1|e2)=9.64663 Lex(e2|e1)=59.48919 Lexical=1 LogCount=4.67283 MVLSASim=NA Monotonic=1 OtherRelated=0.372735 PhrasePenalty=1 RarityPenalty=0 ForwardEntailment=0.177287 SourceTerminalsButNoTarget=0 SourceWords=1 TargetComplexity=0.98821 TargetFormality=0.98464 TargetTerminalsButNoSource=0 TargetWords=1 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=5.00000 WordLogCR=0 ', ' 0-0 ', ' OtherRelated']


,lhs,phrase,paraphrase,features,alignment,entailment
0,[NN],transplant,transplantation,PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -l...,0-0,OtherRelated
1,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,PPDB2.0Score=5.22098 PPDB1.0Score=9.463800 -l...,0-0,Equivalence
2,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,PPDB2.0Score=5.21828 PPDB1.0Score=9.463820 -l...,0-0,Equivalence
3,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,PPDB2.0Score=5.21462 PPDB1.0Score=9.463780 -l...,0-0,Equivalence
4,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,PPDB2.0Score=5.21153 PPDB1.0Score=9.463800 -l...,0-0,Equivalence
...,...,...,...,...,...,...
999995,[NNS],producers,implementers,PPDB2.0Score=3.20078 PPDB1.0Score=18.919140 -...,0-0,Independent
999996,[NNS],developers,mobilizers,PPDB2.0Score=3.20078 PPDB1.0Score=16.644660 -...,0-0,Independent
999997,[NNS],features,arguments,PPDB2.0Score=3.20078 PPDB1.0Score=24.431830 -...,0-0,Independent
999998,[NNS],disagreements,referrals,PPDB2.0Score=3.20078 PPDB1.0Score=24.889380 -...,0-0,Independent


In [19]:
lineCount = 0
with open(file, 'r') as f:
    for line in f:
        lineCount += 1


40138455


In [25]:
from math import *
print(floor(lineCount/1000000))
print(lineCount%1000000)

40
138455


In [32]:
for i in range(0, floor(lineCount/1000000)):
    if i == 0:
        data = parse_file(file, i*1000000, (i+1)*1000000)
        df = pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment'])
    else:
        data = parse_file(file, i*1000000, (i+1)*1000000)
        df = pd.concat([df, pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment'])], ignore_index=True)
if lineCount%1000000 != 0:
    data = parse_file(file, floor(lineCount/1000000)*1000000, lineCount)
    df = pd.concat([df, pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment'])], ignore_index=True)
df.to_csv('ppdb-xxxl-all.csv')    
df